In [54]:
import os 
import tensorflow 
from PIL import Image, ImageDraw
import math
import pandas as pd 
from tqdm import tqdm 
import random 
import shutil

In [14]:
folder_sources = []
Orig_Dataset_path = "Segmented Herbal Leaf images"
New_Dataset_path = "Transformed"

for file in os.listdir(Orig_Dataset_path):
    folder_sources.append(file)
        
print(folder_sources)

['Artocarpus Heterophyllus (Jackfruit)', 'Blumea Balsamifera (Sambong)', 'Citrus Limon (Lemon)', 'Jasminum (Jasmine)', 'Mangifera Indica (Mango)', 'Mentha (Mint)', 'Momordica Charantia (Ampalaya)', 'Moringa Oleifera (Malunggay)', 'Psidium Guajava (Guava)', 'Vitex Negundo (Lagundi)']


In [15]:
for folder in folder_sources:
    f_ = os.path.join(New_Dataset_path, folder)
    if not os.path.exists(f_):
        os.mkdir(f_)
        print("Created Path:", f_)

In [16]:
# Degrees is in radians
def calculate_new_coords(degrees, x, y, iw, ih):
    res_x = iw / 2 + (x - iw / 2) * math.cos(degrees) + (y - ih / 2) * math.sin(degrees)
    res_y = ih / 2 - (x - iw / 2) * math.sin(degrees) + (y - ih / 2) * math.cos(degrees)
    return (res_x, res_y)

In [17]:
BASE_IMAGE_SIZE = ((100, 100))
BASE_CANVAS_SIZE = ((200, 200))
X_VARIATIONS = 5
Y_VARIATIONS = 5
ROTATION_VARIATION = 36
ROTATION_DEGREES = 360 / ROTATION_VARIATION
DRAW_BOUNDING_BOX = False
BORDER_WIDTH = 5 
BORDER_COLOR = "Blue"

In [18]:
for folder in tqdm(folder_sources):
    current_folder_dict = {
        "image_name" : [],
        "x0" : [],
        "y0" : [],
        "x1" : [],
        "y1" : [],
        "x2" : [],
        "y2" : [],
        "x3" : [],
        "y3" : [],
        "class" : []
    }

    for image in os.listdir(os.path.join(Orig_Dataset_path, folder)):
        print(image)

        current_image = Image.open(os.path.join(Orig_Dataset_path, folder, image))
        current_image.thumbnail(BASE_IMAGE_SIZE)
        
        x0, y0 = (0, 0)
        x1, y1 = current_image.size 
        iw, ih = current_image.size 
        
        # Rotate Image
        for i in range(ROTATION_VARIATION + 1):
            current_degrees = int(i * ROTATION_DEGREES)

            rotated_image = current_image.copy()
            rotated_image = rotated_image.rotate(current_degrees, expand=True)

            r_i_width, r_i_height = rotated_image.size

            original_bbox_coords = [(x0, y0), (x1, y0), (x1, y1), (x0, y1)]

            _degrees = math.radians(current_degrees)

            new_coords = []
            _x, _y = 0, 1
            
            for x, y in original_bbox_coords:
                __x, __y = calculate_new_coords(_degrees, x, y, iw, ih)
                _x = min(__x, _x)
                _y = max(__y, _y)
                new_coords.append((__x, __y))

            excess_x = 0 - _x 
            excess_y = r_i_height - _y
            
            final_bbox_coords = [(x + excess_x, y + excess_y) for x, y in new_coords]
                
            # Transposition
            c_w , c_h = BASE_CANVAS_SIZE

            x_steps = abs(r_i_width - c_w) / X_VARIATIONS
            y_steps = abs(r_i_height - c_h) / Y_VARIATIONS
            
            for y in range(Y_VARIATIONS + 1):
                for x in range(X_VARIATIONS + 1):
                    canvas = Image.new("RGB", BASE_CANVAS_SIZE)
                    x_movement, y_movement = int(x_steps) * x, int(y_steps) * y
                    canvas.paste(rotated_image, (x_movement, y_movement))
                    

                    transposed_bbox_coords = [(x + x_movement, y + y_movement) for x, y in final_bbox_coords]

                    if DRAW_BOUNDING_BOX:
                        _d = ImageDraw.Draw(canvas)
                        _d.polygon(transposed_bbox_coords, width=BORDER_WIDTH, outline=BORDER_COLOR)

                    #display(canvas)

                    c0, c1, c2, c3 = transposed_bbox_coords
                    _x0, _y0 = c0 
                    _x1, _y1 = c1
                    _x2, _y2 = c2
                    _x3, _y3 = c3

                    
                    current_folder_dict["x0"].append(_x0 / c_w) 
                    current_folder_dict["y0"].append(_y0 / c_h)
                    current_folder_dict["x1"].append(_x1 / c_w) 
                    current_folder_dict["y1"].append(_y1 / c_h)
                    current_folder_dict["x2"].append(_x2 / c_w) 
                    current_folder_dict["y2"].append(_y2 / c_h)
                    current_folder_dict["x3"].append(_x3 / c_w)
                    current_folder_dict["y3"].append(_y3 / c_h)
                    current_folder_dict["class"].append(folder)
                    
                    image_name = f"{New_Dataset_path}/{folder}/{image}_{current_degrees}_{x}_{y}.jpg"
                    canvas.save(image_name)
                    current_folder_dict["image_name"].append(image_name)

                    # Only 1
                    # break              
                # Only 1
                # break         
            # Only 1 Rotation
            # break
        # Test Only 1 Picture
        # break 
    
    df_current_folder = pd.DataFrame(current_folder_dict)
    print(df_current_folder.head())
    df_current_folder.to_csv(f"{New_Dataset_path}/{folder}/annotations.csv")

    # Test Only 1 Folder
    # break 


  0%|          | 0/10 [00:00<?, ?it/s]

1.jpg
10.jpg
11.jpg
12.jpg
13.jpg
14.jpg
15.jpg
16.jpg
17.jpg
18.jpg
19.jpg
2.jpg
20.jpg
21.jpg
22.jpg
23.jpg
24.jpg
25.jpg
26.jpg
27.jpg
28.jpg
29.jpg
3.jpg
30.jpg
31.jpg
32.jpg
33.jpg
34.jpg
35.jpg
36.jpg
37.jpg
38.jpg
39.jpg
4.jpg
40.jpg
41.jpg
42.jpg
43.jpg
44.jpg
45.jpg
46.jpg
47.jpg
48.jpg
49.jpg
5.jpg
50.jpg
51.jpg
52.jpg
6.jpg
7.jpg
8.jpg
9.jpg
                                          image_name     x0   y0     x1   y1  \
0  Transformed/Artocarpus Heterophyllus (Jackfrui...  0.000  0.0  0.375  0.0   
1  Transformed/Artocarpus Heterophyllus (Jackfrui...  0.125  0.0  0.500  0.0   
2  Transformed/Artocarpus Heterophyllus (Jackfrui...  0.250  0.0  0.625  0.0   
3  Transformed/Artocarpus Heterophyllus (Jackfrui...  0.375  0.0  0.750  0.0   
4  Transformed/Artocarpus Heterophyllus (Jackfrui...  0.500  0.0  0.875  0.0   

      x2   y2     x3   y3                                 class  
0  0.375  0.5  0.000  0.5  Artocarpus Heterophyllus (Jackfruit)  
1  0.500  0.5  0.125  0.5  Artoc

 10%|█         | 1/10 [01:52<16:51, 112.34s/it]

1.jpg
10.jpg
11.jpg
12.jpg
13.jpg
14.jpg
15.jpg
16.jpg
17.jpg
18.jpg
19.jpg
2.jpg
20.jpg
21.jpg
22.jpg
23.JPG
24.jpg
25.jpg
26.jpg
27.jpg
28.jpg
29.jpg
3.jpg
30.JPG
31.jpg
32.jpg
33.jpg
34.jpg
35.jpg
36.JPG
37.jpg
38.JPG
39.JPG
4.jpg
40.jpg
41.jpg
42.jpg
43.jpg
44.jpg
45.jpg
46.jpg
47.jpg
48.jpg
49.jpg
5.jpg
50.JPG
51.jpg
52.jpg
53.jpg
6.jpg
7.jpg
8.jpg
9.jpg
                                          image_name   x0   y0   x1   y1   x2  \
0  Transformed/Blumea Balsamifera (Sambong)/1.jpg...  0.0  0.0  0.5  0.0  0.5   
1  Transformed/Blumea Balsamifera (Sambong)/1.jpg...  0.1  0.0  0.6  0.0  0.6   
2  Transformed/Blumea Balsamifera (Sambong)/1.jpg...  0.2  0.0  0.7  0.0  0.7   
3  Transformed/Blumea Balsamifera (Sambong)/1.jpg...  0.3  0.0  0.8  0.0  0.8   
4  Transformed/Blumea Balsamifera (Sambong)/1.jpg...  0.4  0.0  0.9  0.0  0.9   

      y2   x3     y3                         class  
0  0.375  0.0  0.375  Blumea Balsamifera (Sambong)  
1  0.375  0.1  0.375  Blumea Balsamifera (Sam

 20%|██        | 2/10 [03:28<13:43, 102.88s/it]

...

 80%|████████  | 8/10 [12:54<03:17, 98.93s/it]

1.jpg
10.JPG
11.jpg
12.jpg
13.jpg
14.jpg
15.jpg
16.jpg
17.jpg
18.JPG
19.jpg
2.jpg
20.jpg
21.jpg
22.jpg
23.jpg
24.jpg
25.jpg
26.jpg
27.jpg
28.jpg
29.jpg
3.jpg
30.jpg
31.jpg
32.jpg
33.jpg
34.jpg
35.jpg
36.jpg
37.jpg
38.jpg
39.jpg
4.jpg
40.jpg
41.jpg
42.jpg
43.jpg
44.jpg
45.jpg
46.jpg
47.jpg
48.jpg
49.jpg
5.jpg
50.jpg
51.jpg
52.jpg
53.jpg
6.jpg
7.jpg
8.jpg
9.jpg
                                          image_name   x0   y0   x1   y1   x2  \
0  Transformed/Psidium Guajava (Guava)/1.jpg_0_0_...  0.0  0.0  0.5  0.0  0.5   
1  Transformed/Psidium Guajava (Guava)/1.jpg_0_1_...  0.1  0.0  0.6  0.0  0.6   
2  Transformed/Psidium Guajava (Guava)/1.jpg_0_2_...  0.2  0.0  0.7  0.0  0.7   
3  Transformed/Psidium Guajava (Guava)/1.jpg_0_3_...  0.3  0.0  0.8  0.0  0.8   
4  Transformed/Psidium Guajava (Guava)/1.jpg_0_4_...  0.4  0.0  0.9  0.0  0.9   

      y2   x3     y3                    class  
0  0.335  0.0  0.335  Psidium Guajava (Guava)  
1  0.335  0.1  0.335  Psidium Guajava (Guava)  
2  0.33

 90%|█████████ | 9/10 [14:23<01:35, 95.65s/it]

1.jpg
10.jpg
11.jpg
12.jpg
13.jpg
14.jpg
15.jpg
16.jpg
17.jpg
18.jpg
19.jpg
2.jpg
20.jpg
21.jpg
22.jpg
23.jpg
24.jpg
25.jpg
26.jpg
27.jpg
28.jpg
29.jpg
3.jpg
30.jpg
31.jpg
32.jpg
33.jpg
34.jpg
35.jpg
36.jpg
37.jpg
38.jpg
39.jpg
4.jpg
40.jpg
41.jpg
42.jpg
43.jpg
44.jpg
45.jpg
46.jpg
47.jpg
48.jpg
49.jpg
5.jpg
50.jpg
51.jpg
52.jpg
53.jpg
6.jpg
7.jpg
8.jpg
9.jpg
                                          image_name   x0   y0   x1   y1   x2  \
0  Transformed/Vitex Negundo (Lagundi)/1.jpg_0_0_...  0.0  0.0  0.5  0.0  0.5   
1  Transformed/Vitex Negundo (Lagundi)/1.jpg_0_1_...  0.1  0.0  0.6  0.0  0.6   
2  Transformed/Vitex Negundo (Lagundi)/1.jpg_0_2_...  0.2  0.0  0.7  0.0  0.7   
3  Transformed/Vitex Negundo (Lagundi)/1.jpg_0_3_...  0.3  0.0  0.8  0.0  0.8   
4  Transformed/Vitex Negundo (Lagundi)/1.jpg_0_4_...  0.4  0.0  0.9  0.0  0.9   

     y2   x3    y3                    class  
0  0.38  0.0  0.38  Vitex Negundo (Lagundi)  
1  0.38  0.1  0.38  Vitex Negundo (Lagundi)  
2  0.38  0.2 

100%|██████████| 10/10 [15:50<00:00, 95.09s/it]


In [62]:
x = []
for folder in tqdm(os.listdir(New_Dataset_path)):
    train_folder = os.path.join(New_Dataset_path, folder, "train")
    test_folder = os.path.join(New_Dataset_path, folder, "test")

    if os.path.exists(train_folder):
        continue

    print(folder)
    current_annotation_csv = list(filter(lambda x: "csv" in x, os.listdir(os.path.join(New_Dataset_path, folder))))[0]
    #print(current_annotation_csv)

    raw_df = pd.read_csv(os.path.join(New_Dataset_path, folder, current_annotation_csv))
    current_image_size = len(raw_df["image_name"])
    train_size = int(0.8 * current_image_size)
    
    df = raw_df.copy()

    train_df = df.sample(train_size)
    test_df = df.drop(train_df.index, axis=0)
    
    #print(len(train_df), train_size, len(test_df))


    
    
    if not os.path.exists(train_folder):
        os.mkdir(train_folder)

    if not os.path.exists(test_folder):
        os.mkdir(test_folder)

    for file in train_df["image_name"].tolist():
        image_file = file.split("/")[-1]
        new_image_path = os.path.join(train_folder, image_file)
        shutil.move(file, new_image_path)

    train_df["image_name"] = train_df["image_name"].map(lambda x: os.path.join(train_folder, x.split("/")[-1]))
    train_df.to_csv(os.path.join(New_Dataset_path, folder, "train_annotations.csv"))
    
    

    for file in test_df["image_name"].tolist():
        image_file = file.split("/")[-1]
        new_image_path = os.path.join(test_folder, image_file)

        shutil.move(file, new_image_path)
    
    test_df["image_name"] = test_df["image_name"].map(lambda x: os.path.join(test_folder, x.split("/")[-1]))
    test_df.to_csv(os.path.join(New_Dataset_path, folder, "test_annotations.csv"))

    
    
    

  0%|          | 0/10 [00:00<?, ?it/s]

Blumea Balsamifera (Sambong)


 20%|██        | 2/10 [03:54<15:36, 117.04s/it]

Citrus Limon (Lemon)


 30%|███       | 3/10 [05:10<11:40, 100.10s/it]

...

 80%|████████  | 8/10 [12:40<03:16, 98.24s/it]

Psidium Guajava (Guava)


 90%|█████████ | 9/10 [13:58<01:32, 92.12s/it]

Vitex Negundo (Lagundi)


100%|██████████| 10/10 [15:17<00:00, 91.70s/it]


In [25]:
_ = 0
for df in x:
    print(len(df))
    _ += len(df)

print(_)

69264
70596
66600
69264
69264
65268
66600
95904
70596
70596
713952


In [26]:
X = pd.concat(x)

In [27]:
X

,Unnamed: 0,image_name,x0,y0,x1,y1,x2,y2,x3,y3,class
0,0,Transformed/Artocarpus Heterophyllus (Jackfrui...,0.000,0.0,0.375,0.0,0.375,0.5,0.000,0.5,Artocarpus Heterophyllus (Jackfruit)
1,1,Transformed/Artocarpus Heterophyllus (Jackfrui...,0.125,0.0,0.500,0.0,0.500,0.5,0.125,0.5,Artocarpus Heterophyllus (Jackfruit)
2,2,Transformed/Artocarpus Heterophyllus (Jackfrui...,0.250,0.0,0.625,0.0,0.625,0.5,0.250,0.5,Artocarpus Heterophyllus (Jackfruit)
3,3,Transformed/Artocarpus Heterophyllus (Jackfrui...,0.375,0.0,0.750,0.0,0.750,0.5,0.375,0.5,Artocarpus Heterophyllus (Jackfruit)
4,4,Transformed/Artocarpus Heterophyllus (Jackfrui...,0.500,0.0,0.875,0.0,0.875,0.5,0.500,0.5,Artocarpus Heterophyllus (Jackfruit)
...,...,...,...,...,...,...,...,...,...,...,...
70591,70591,Transformed/Vitex Negundo (Lagundi)/9.jpg_360_...,0.100,0.5,0.600,0.5,0.600,1.0,0.100,1.0,Vitex Negundo (Lagundi)
70592,70592,Transformed/Vitex Negundo (Lagundi)/9.jpg_360_...,0.200,0.5,0.700,0.5,0.700,1.0,0.200,1.0,Vitex Negundo (Lagundi)
70593,70593,Transformed/Vitex Negundo (Lagundi)/9.jpg_360_...,0.300,0.5,0.800,0.5,0.800,1.0,0.300,1.0,Vitex Negundo (Lagundi)
70594,70594,Transformed/Vitex Negundo (Lagundi)/9.jpg_360_...,0.400,0.5,0.900,0.5,0.900,1.0,0.400,1.0,Vitex Negundo (Lagundi)


In [28]:
X.to_csv("Full Dataset Annotations.csv")